# 模型构建

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


## 定义模型A

In [2]:
# 定义模型A
class ModelA(nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv3d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



## 定义模型B

In [3]:
# 定义模型B
class ModelB(nn.Module):
    def __init__(self):
        super(ModelB, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(7, 64, kernel_size=3, padding=1),  # 7 channels: 3 for velocity, 4 for abcd
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv3d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x, abcd):
        x = torch.cat([x, abcd], dim=1)  # Concatenate velocity and abcd inputs
        x = self.encoder(x)
        x = self.decoder(x)
        return x

##  示例训练循环（适用于模型A和模型B）

In [4]:

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)

## 定义CUDA

In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

##  实例化模型

In [6]:
model_a = ModelA().to(device)
model_b = ModelB().to(device)

## 定义损失和优化器

In [7]:
criterion = nn.MSELoss()
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001)
optimizer_b = optim.Adam(model_b.parameters(), lr=0.001)

## 定义数据类

In [8]:
from torch.utils.data import Dataset, DataLoader

In [9]:
class VelocityDataset(Dataset):
    def __init__(self, velocity_files, abcd_files=None, mask_ratio=0.2):
        """
        velocity_files: list of paths to velocity npy files
        abcd_files: list of paths to abcd npy files (only for Model B)
        mask_ratio: percentage of the velocity field to mask (e.g., 0.2 means 20% of the field will be masked)
        """
        self.velocity_files = velocity_files
        self.abcd_files = abcd_files
        self.mask_ratio = mask_ratio

    def __len__(self):
        return len(self.velocity_files)

    def __getitem__(self, idx):
        # 加载速度场
        velocity = np.load(self.velocity_files[idx])

        # 生成掩码
        mask = np.random.rand(*velocity.shape) < self.mask_ratio
        masked_velocity = velocity.copy()
        masked_velocity[mask] = 0  # 掩盖部分速度场数据

        # 转换为PyTorch张量
        masked_velocity = torch.from_numpy(masked_velocity).float()
        velocity = torch.from_numpy(velocity).float()

        if self.abcd_files is not None:
            # 加载并转换物理量 a, b, c, d
            abcd = np.load(self.abcd_files[idx])
            abcd = torch.from_numpy(abcd).float()
            return masked_velocity, abcd, velocity  # 返回模型B的输入和目标
        else:
            return masked_velocity, velocity  # 返回模型A的输入和目标


### 获取训练和测试集的文件列表

In [11]:
import os

In [12]:
train_velocity_files = [os.path.join('data/processed/train/velocities', f) for f in os.listdir('data/processed/train/velocities')]
train_abcd_files = [os.path.join('data/processed/train/abcd', f) for f in os.listdir('data/processed/train/abcd')]

test_velocity_files = [os.path.join('data/processed/test/velocities', f) for f in os.listdir('data/processed/test/velocities')]
test_abcd_files = [os.path.join('data/processed/test/abcd', f) for f in os.listdir('data/processed/test/abcd')]

### 创建Dataset实例

In [13]:

train_dataset_a = VelocityDataset(train_velocity_files, mask_ratio=0.2)
train_dataset_b = VelocityDataset(train_velocity_files, train_abcd_files, mask_ratio=0.2)

test_dataset_a = VelocityDataset(test_velocity_files, mask_ratio=0.2)
test_dataset_b = VelocityDataset(test_velocity_files, test_abcd_files, mask_ratio=0.2)


### 创建DataLoader实例

In [24]:

train_loader_a = DataLoader(train_dataset_a, batch_size=2, shuffle=True, num_workers=0)
train_loader_b = DataLoader(train_dataset_b, batch_size=2, shuffle=True, num_workers=0)

test_loader_a = DataLoader(test_dataset_a, batch_size=4, shuffle=False, num_workers=0)
test_loader_b = DataLoader(test_dataset_b, batch_size=4, shuffle=False, num_workers=0)


In [25]:
import numpy as np

In [26]:
num_epochs = 5  # 设置训练的轮数

for epoch in range(num_epochs):
    train_loss_a = train(model_a, train_loader_a, optimizer_a, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Model A Loss: {train_loss_a:.4f}")


KeyboardInterrupt: 